In [ ]:
'''@author: osamah Abdelhaq last edit date: 7/26/2020'''

# Data Transformation + Visualization

In [ ]:
# !pip install -q -U watermark
# !pip install -qq transformers
# %reload_ext watermark
# %watermark -v -p numpy,pandas,torch,transformers

In [ ]:
import pandas as pd
#@title Setup & Config
# import transformers
# from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
# import torch

import numpy as np
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
# from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
# from collections import defaultdict
# from textwrap import wrap

# from torch import nn, optim
# from torch.utils.data import Dataset, DataLoader
# import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

# RANDOM_SEED = 42
# np.random.seed(RANDOM_SEED)
# torch.manual_seed(RANDOM_SEED)

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device

In [ ]:
#importing all data
reddit_USPS = pd.read_pickle('reddit_USPS')
reddit_ppp = pd.read_pickle('reddit_ppp')
reddit_stim = pd.read_pickle('reddit_stim')
reddit_une = pd.read_pickle('reddit_une')

tiktok_USPS = pd.read_pickle('tiktok_USPS')
tiktok_stim = pd.read_pickle('tiktok_stim')
tiktok_une = pd.read_pickle('tiktok_une')

fb_USPS = pd.read_csv('facebook_usps_frame.csv',index_col=0)
fb_IRS = pd.read_csv('facebook_irs_frame.csv',index_col=0)
fb_une = pd.read_csv('facebook_unemployment_frame.csv',index_col=0)

In [ ]:
'''Editing date to be of format month/day/year'''

#using pandas replace function
dates = tiktok_USPS['date'].replace(['5d ago','4d ago','3d ago','2d ago','1d ago'],['6-8','6-9','6-10','6-11','6-12'])

#using for loop to remedy other possible issues
for index, date in enumerate(dates):
    if 'ago' in date:
        dates[index] = '6/12/2020'
    elif 'K' in date:
        dates[index] = '4/27/2020'
    else:
        #replacing dashes with slashes
        date = date.replace('-' , '/') 
        #adding year
        dates[index] = date+'/2020'

tiktok_USPS['date'] = dates

tiktok_USPS['date'] = pd.to_datetime(tiktok_USPS.date)

#repeat for next use case
dates = tiktok_stim['date'].replace(['5d ago','4d ago','3d ago','2d ago','1d ago'],['6/8','6/9','6/10','6/11','6/12'])

for index, date in enumerate(dates):
    if 'ago' in date:
        dates[index] = '6/12/2020'
    elif 'K' in date:
        dates[index] = '4/27/2020'
    else:
        date = date.replace('-' , '/') 
        dates[index] = date+'/2020'

tiktok_stim['date'] = dates

tiktok_stim['date'] = pd.to_datetime(tiktok_stim.date)

#repeat for next use case
dates = tiktok_une['date'].replace(['5d ago','4d ago','3d ago','2d ago','1d ago'],['6/8','6/9','6/10','6/11','6/12'])

for index, date in enumerate(dates):
    if 'ago' in date:
        dates[index] = '6/12/2020'
    elif 'K' in date:
        dates[index] = '4/27/2020'
    else:
        date = date.replace('-' , '/') 
        dates[index] = date+'/2020'

tiktok_une['date'] = dates
tiktok_une['date'] = pd.to_datetime(tiktok_une.date)

In [ ]:
#renaming columns for ease of stacking
tiktok_USPS.rename(columns = {'text':'Comments'}, inplace = True) 
tiktok_stim.rename(columns = {'text':'Comments'}, inplace = True)
tiktok_une.rename(columns = {'text':'Comments'}, inplace = True)

tiktok_une.head()

In [ ]:
'''Combining facebook data with tiktok data for polarity over time purposes'''
#aligning data to be uniform
fb_irs = fb_IRS.drop(['like','reply_to','day','month','dayofweek','subjectivity_label','polarity_label'], axis=1)
fb_usps = fb_USPS.drop(['like','reply_to','day','month','dayofweek','subjectivity_label','polarity_label'], axis=1)
fb_une = fb_une.drop(['like','reply_to','day','month','dayofweek','subjectivity_label','polarity_label'],axis=1)

#renaming columns to match tiktok data set
fb_irs.rename(columns = {'text':'Comments'}, inplace = True) 
fb_usps.rename(columns = {'text':'Comments'}, inplace = True) 
fb_une.rename(columns = {'text':'Comments'}, inplace = True) 

fb_irs.rename(columns = {'text_prep':'comment_ready'}, inplace = True) 
fb_usps.rename(columns = {'text_prep':'comment_ready'}, inplace = True) 
fb_une.rename(columns = {'text_prep':'comment_ready'}, inplace = True) 

columns_titles = ["name","Comments","comment_ready","date","polarity", "subjectivity"]

fb_irs=fb_irs.reindex(columns=columns_titles)
fb_usps=fb_usps.reindex(columns=columns_titles)
fb_une=fb_une.reindex(columns=columns_titles)

fb_une.head()

In [ ]:
# fb_une.to_csv('fb_une_all.csv')
# tiktok_une.to_csv('tiktok_une_all.csv')
# fb_irs.to_csv('fb_irs_all.csv')
# tiktok_stim.to_csv('tiktok_irs_all.csv')
# tiktok_USPS.to_csv('tiktok_usps_all.csv')
# fb_usps.to_csv('fb_usps_all.csv')

In [ ]:
#stacking tiktok and facebook by use case
une_frames = [fb_une, tiktok_une]
usps_frames = [fb_usps, tiktok_USPS]
irs_frames = [fb_irs,tiktok_stim]

usps_data = pd.concat(usps_frames)
une_data = pd.concat(une_frames)
irs_data = pd.concat(irs_frames)

usps_data['date'] = pd.to_datetime(usps_data['date'])
une_data['date'] = pd.to_datetime(une_data['date'])
irs_data['date'] = pd.to_datetime(irs_data['date'])

In [ ]:
#***excludes reddit data for lack of date
plt.figure(figsize=(20,5))
ax = sns.lineplot(x = 'date', y = 'polarity', data = usps_data, ci=None)
ax = sns.lineplot(x = 'date', y = 'polarity', data=irs_data, ci=None)
ax = sns.lineplot(x = 'date', y = 'polarity', data=une_data, ci=None)
ax.legend(labels=["USPS","IRS","Unemployment"])
ax.set_title("Average Polarity over Time")
ax.set_xlim(irs_data['date'].min(), irs_data['date'].max())


#USPS polarity is more volatile

'''Polarity is limited by lack of hashtag, gif, and external links interpretation'''
print('**Important note: only a small number of USPS march comments, accounting for the polarity spike.')

In [ ]:
une_data.head()
une_data.to_csv('fb_tiktok_une.csv')

In [ ]:
#trying to identify usps_data outliers in polarization
#uncomment code to print it all

# dates = usps_data.date.to_numpy()
# # for i in usps_data['date']:
# #   if '2020-03' in i:
# #     print(i)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# print(usps_data.loc[usps_data.polarity==1])

#from seeing all march comments with polarity = 1.0, we can see that there are a limited number of samples (<5)

In [ ]:
#preparing to combine datasets by use case with reddit datasets for topic modeling
irs_data = irs_data.drop(['name','date'], axis=1)
usps_data = usps_data.drop(['name','date'], axis=1)
une_data = une_data.drop(['name','date'],axis=1)
irs_data.head()

In [ ]:
#confirming they match
reddit_stim.head()

In [ ]:
reddit_stim.to_csv('reddit_stimulus_all.csv')
reddit_ppp.to_csv('reddit_paycheck_protection_all.csv')
reddit_USPS.to_csv('reddit_usps_all.csv')
reddit_une.to_csv('reddit_une_all.csv')

In [ ]:
#combining datasets for each use case

usps_frames = [reddit_USPS, usps_data] 
irs_frames = [reddit_ppp,irs_data, reddit_stim] 
une_frames = [une_data, reddit_une] 

usps_data = pd.concat(usps_frames)
irs_data = pd.concat(irs_frames)
une_data = pd.concat(une_frames)

In [ ]:
# irs_data.to_csv(r'C:\Users\osama\Desktop\irs_data.csv')
# usps_data.to_csv(r'C:\Users\osama\Desktop\usps_data.csv')
# une_data.to_csv(r'C:\Users\osama\Desktop\une_data.csv')

In [ ]:
#view basic information
print('IRS Info:',irs_data.info()) #7,291 comments
print('\nUSPS Info:',usps_data.info()) #11,137 comments
print('\nUnemployment Info:',une_data.info()) #4,776 comments

samp=len(irs_data)+len(usps_data)+len(une_data)

print('\nTotal Number of Comments =', samp)

In [ ]:
#finding where text preprocessing created null values and replacing with original comment
irs_nulls = np.where(pd.isnull(irs_data['comment_ready']))
usps_nulls = np.where(pd.isnull(usps_data['comment_ready']))
une_nulls = np.where(pd.isnull(une_data['comment_ready']))

#for loop to replace null with original via index
for i in une_nulls[0]:
    comm = une_data['Comments'].iloc[i]
    une_data['comment_ready'].iloc[i] = comm 

#for loop to replace null with original via index
for i in usps_nulls[0]:
    comm = usps_data['Comments'].iloc[i]
    usps_data['comment_ready'].iloc[i] = comm 

#for loop to replace null with original via index
for i in irs_nulls[0]:
    comm = irs_data['Comments'].iloc[i]
    irs_data['comment_ready'].iloc[i] = comm 

# Semi-Supervised Topic Modeling Using Correlation Explanation

Step 1: Conduct Unsupervised model and examine topic, testing both large topic count and small topic count

Step 2: Anchor Words to topics after the Unsupervised model

Step 3: Experiment with anchor strength parameter and topic count parameter

Step 4: Add to Dataframe

**Keep in mind, no topic model will be 100% perfect, but a semi-supervised approach will greatly improve results


In [ ]:
#necessary package
!pip install corextopic

### Unemployment Data


In [ ]:
#term frequency–inverse document frequency

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(une_data['comment_ready'])
tfidf = vectorizer.transform(une_data['comment_ready'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

#### Unsupervised

In [ ]:
#testing with 20 topics
from corextopic import corextopic as ct
model = ct.Corex(n_hidden=20)
model = model.fit(
    tfidf,
    words=vocab
)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
#testing with 5 topics
model = ct.Corex(n_hidden=5)
model = model.fit(
    tfidf,
    words=vocab
)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
#testing with 10 topics
model = ct.Corex(n_hidden=10)
model = model.fit(
    tfidf,
    words=vocab
)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
#testing with 15 topics
model = ct.Corex(n_hidden=15)
model = model.fit(
    tfidf,
    words=vocab
)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

#### Semi-Supervised

In [ ]:
# Anchors designed to nudge the model towards measuring specific topics
anchors = [
    ["march", "economic", "opportunity", "florida"], #FLDEO
    ["submit", "question", "application"], #application questions
    ["payment", "receive", "600"], #unemployment payment
    ["eligible", "pending", "waiting", "file", "eligible", "still"], #eligibility
    ["claim","received","week"], #claim received
    ["deposit", "org", "floridajobs"], #floridajobs
    ["call", "account", "two", "time", "information"], #customer service calls
    ["covid", "virus"], #covid
    ["benefit","unemployment","minimum wage"], #minimum wage, unemployment benefits
    ["care","desantis","please","fix","help","act"], #Florida CARES Act/Governor Desantis
    ["work","search","company"], #job search
    ["bank","check"], #direct deposit
    ["claim", "weeks", "able"], #delayed claims
    ["election", "trump", "obama"], #president, stock market
    ["google"], #google, algorithm resume search
    ["stimulus"] #phone customer service
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=18)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=4 # Tell the model how much it should rely on the anchors
)

for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
topic_1 = 'fldeo'
topic_2 = 'application questions'
topic_3 = 'unemployment payment'
topic_4 = 'eligibility'
topic_5 = 'claim received'
topic_6 = 'Florida Jobs'
topic_7 = 'customer service'
topic_8 = 'covid corona virus'
topic_9 = 'unemployment benefits minimum wage'
topic_10 = 'florida cares act governor desantis'
topic_11 = 'job search'
topic_12 = 'direct deposit'
topic_13 = 'delayed claims'
topic_14 = 'president Obama Trump Stock Market'
topic_15 = 'google algorithm'
topic_16 = 'phone customer servce'
topic_17 = 'etc.'
topic_18 = 'money payment'
topic_19 = 'unidentified'

topics = [topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,\
          topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18]

In [ ]:
#making new dataframe of topics to visualize data
topic_une = pd.DataFrame(
    model.transform(tfidf), 
    columns=topics
).astype(int)
topic_une.index = une_data.index
df_une = pd.concat([une_data.Comments, topic_une], axis=1)

In [ ]:
print(df_une)

In [ ]:
sns.heatmap(topic_une)

In [ ]:
#counting amount of each topic for visualization
counts = topic_une.sum(axis=0).to_numpy()
# uni = topic_une.sum(axis=1).to_numpy()
# uni = np.where(uni!=0,100,0)
# uni = np.where(uni==0,1,uni)
# uni = np.where(uni==100,0,uni)
# uni_count = np.where(uni==0)
# counts = np.append(counts,len(uni_count[0]))
une_topic_counts = counts
# topics = [topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,\
#           topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19]
df_counts = pd.DataFrame({'Topic':topics,'Count':une_topic_counts})
print('**Total number of comments:', samp)
sns.barplot(x='Count',y='Topic',data=df_counts)

# #important to note that sentences have multiple topics, there are misclassifications as well

In [ ]:
#using the last data frame to substitute the binary values with the actual topics
#then appending it to the master data frame as a new column
test = list(
    model.transform(tfidf)
)

for i in range(len(test)):
    tops = np.where(test[i]==True)
    individual_topics = []
    for j in tops[0]:
        it = topics[j]
        individual_topics.append(it)
    test[i]=individual_topics

une_data['topics']=test
nested_topics = une_data['topics']
topics_string = []
for i in une_data['topics']:
    topics_string.append(', '.join(i))
une_data['topics'] = topics_string

une_data.head()

In [ ]:
# #substituting classifications for polarity values
topic_une['polarity'] = une_data['polarity']

#counting each topic by positive, neutral, negative polarities
positives = []

for i in (topics):
    pos = topic_une.loc[(topic_une[i] == 1) & (topic_une['polarity']>0.0)]
    positives.append(pos.shape[0])

negatives = []

for i in (topics):
    neg = topic_une.loc[(topic_une[i] == 1) & (topic_une['polarity']<0.0)]
    negatives.append(neg.shape[0])

neutral = []

for i in topics:
    net = topic_une.loc[(topic_une[i] == 1) & (topic_une['polarity']>-0.01) & (topic_une['polarity']<0.01)]
    neutral.append(net.shape[0])

In [ ]:
topic_polarity_matrix=pd.DataFrame({'Topics':topics,'positive':positives,'neutral':neutral,'negative':negatives,'count':une_topic_counts})
topic_polarity_matrix.head()

In [ ]:
import plotly.graph_objects as go

x=topic_polarity_matrix.Topics
fig = go.Figure(go.Bar(x=x, y=topic_polarity_matrix.positive, name='Positive Polarity'))
fig.add_trace(go.Bar(x=x, y=topic_polarity_matrix.neutral, name='Neutral Polarity'))
fig.add_trace(go.Bar(x=x, y=topic_polarity_matrix.negative, name='Negative Polarity'))

fig.update_layout(barmode='stack')
fig.show()

In [ ]:
print('Bivariate Distribution: ')
sns.jointplot(x=une_data.polarity, y=une_data.subjectivity);

In [ ]:
from nltk.tokenize.casual import TweetTokenizer
from nltk import ngrams, word_tokenize, pos_tag
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from itertools import chain #to unnest lists
from wordcloud import WordCloud, ImageColorGenerator,STOPWORDS
from nltk.corpus import stopwords, wordnet
import gensim
from gensim import matutils, models, corpora
import scipy.sparse
t = TweetTokenizer()
def prep(frame):
    '''input: column of dataframe
    output: returns nested list of tokenized, lemmatized words with stop words removed'''

    stop_words = set(stopwords.words('english'))
    lem = nltk.WordNetLemmatizer()
    
    #lemmatize words
    #lowercase words
    #remove stop words
    tokens = []
    for i in frame:
        sentence = []
        
        #lemmatize based on the pos
        # Get the single character pos constant from pos_tag like this:
        tik = t.tokenize(i)
        for tok in tik:
            pos_label = (pos_tag(tok)[0][1][0]).lower()

            # pos_refs = {'n': ['NN', 'NNS', 'NNP', 'NNPS'],
            #            'v': ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'],
            #            'r': ['RB', 'RBR', 'RBS'],
            #            'a': ['JJ', 'JJR', 'JJS']}
            
            if pos_label == 'j': pos_label = 'a'    # 'j' <--> 'a' reassignment

            if pos_label in ['r']:  # For adverbs it's a bit different
                try:
                    s = wordnet.synset(tok+'.r.1').lemmas()[0].pertainyms()[0].name()
                except:
                    s = tok
                    continue

            elif pos_label in ['a', 's', 'v']: # For adjectives and verbs
                try:
                    s = lem.lemmatize(tok, pos=pos_label)
                except:
                    s = tok
                    continue

            else:   # For nouns and everything else as it is the default kwarg
                try:
                    s = lem.lemmatize(tok)
                except:
                    s = tok
                    continue

            if s not in stop_words:
                #tokenize
                sentence.append(s)
            else:
                del s
            
        tokens.append(sentence)

    return tokens

def alph(frame):      
    '''input: takes a nested list of tokenized words
    output: returns unnested list of tokens with letters, emojis, and punctuation removed.
    Only alphabetical characters greater than length 4 are returned'''
    #unnest list
    unlist = list(chain(*frame))

    #remove anything non alphabetical and append to une_alph
    regex = re.compile('[^a-zA-Z]')
    
    alph = []
    
    #only keep words greater than 4 letters long
    for i in unlist:
        if (len(i) > 4 and i != ", "):
            alph.append(regex.sub('', i))
            
    return alph

def freqChart(alph):
    '''Input: word list, not nested, of only alphabetical words
    Output: returns word frequency list(l) and graph of top 20 words (f)
    returns least common words (u)
    '''
    
    #create frequency distribution of words
    fdist1 = FreqDist(alph)
    
    #ten most common words
    l = fdist1.most_common(10)
    
    #frequency distribution chart of top 20 words
    f = fdist1.plot(20, cumulative=True)
    
    #10 most infrequent words
    u = fdist1.hapaxes()
    
    return l, u[:10], f

def plot_word_cloud(alph):
    '''input: word list, not nested, of only alphabetical words
    output: plots word cloud'''
    str_Cloud = " ".join(alph)
    
    wc = WordCloud(# font_path='simsun.ttf',  
               background_color="black",  
               width=600,height=300,
               min_font_size=10,
               stopwords=STOPWORDS,
               max_words=2000,  
               # mask=back_coloring, 
               max_font_size=120, 
               random_state=42,
               collocations=False,
               colormap = 'cool').generate(str_Cloud)
    # image_colors = ImageColorGenerator(back_coloring)
    plt.figure(figsize=(60,30))
    plt.imshow(wc)
    plt.axis("off")
    plt.savefig("word_cloud.jpeg")
    plt.show()

In [ ]:
'''word frequency analysis'''
#tokenize, lowercase, lemmatize, remove stop words
une_tokens = prep(une_data['Comments'])

#add tokens as a new column ('text_prep') to dataframe
# unemployment_frame['text_prep'] = une_tokens

#remove anything not alphabetical
une_alph = alph(une_tokens)

#fetch word frequencies
une_freq_list, une_not_freq_list, une_freq_chart = freqChart(une_alph)

In [ ]:
#print results
print(une_freq_chart)
print('Top Ten Most Common Words:',une_freq_list)
print('Ten Least Common Words:',une_not_freq_list)
# print('lexical diversity: ',lexical_diversity(une_alph))

In [ ]:
'''word frequency analysis with bigrams'''

#repeat but with bigrams for more insight
une_bigrams = list(nltk.bigrams(une_alph))
une_bigrams_freq_list, une_bigrams_uncommon, une_bigrams_graph = freqChart(une_bigrams)
print(une_bigrams_graph)

In [ ]:
'''word frequency analysis with trigrams'''

#repeat but with trigrams for more insight
une_trigrams = list(nltk.trigrams(une_alph))
une_trigrams_freq_list, une_trigrams_uncommon, une_trigrams_graph = freqChart(une_trigrams)
print(une_trigrams_graph)

In [ ]:
'''word cloud'''

plot_word_cloud(une_alph)

In [ ]:
# une_data.to_csv(r'une_data.csv')

### USPS Data

In [ ]:
vectorizer = vectorizer.fit(usps_data['comment_ready'])
tfidf = vectorizer.transform(usps_data['comment_ready'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

In [ ]:
#20 topics
model = ct.Corex(n_hidden=20)
model = model.fit(
    tfidf,
    words=vocab
)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
model = ct.Corex(n_hidden=10)
model = model.fit(
    tfidf,
    words=vocab
)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
model = ct.Corex(n_hidden=15)
model = model.fit(
    tfidf,
    words=vocab
)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

### IRS Data

In [ ]:
vectorizer = vectorizer.fit(irs_data['comment_ready'])
tfidf = vectorizer.transform(irs_data['comment_ready'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

In [ ]:
#20 topics
model = ct.Corex(n_hidden=20)
model = model.fit(
    tfidf,
    words=vocab
)
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

# Document Summarization

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

def _create_frequency_table(text_string) -> dict:

    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable

In [ ]:
def _score_sentences(sentences, freqTable) -> dict:
    """
    score a sentence by its words
    Basic algorithm: adding the frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = dict()

    for sentence in sentences:
        word_count_in_sentence = (len(word_tokenize(sentence)))
        word_count_in_sentence_except_stop_words = 0
        for wordValue in freqTable:
            if wordValue in sentence.lower():
                word_count_in_sentence_except_stop_words += 1
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    sentenceValue[sentence[:10]] = freqTable[wordValue]

        if sentence[:10] in sentenceValue:
            sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] / word_count_in_sentence_except_stop_words

        '''
        Notice that a potential issue with our score algorithm is that long sentences will have an advantage over short sentences. 
        To solve this, we're dividing every sentence score by the number of words in the sentence.
        
        Note that here sentence[:10] is the first 10 character of any sentence, this is to save memory while saving keys of
        the dictionary.
        '''

    return sentenceValue


In [ ]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original text
    average = (sumValues / len(sentenceValue))

    return average

In [ ]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [ ]:
une_comments = []
for comment in une_data.Comments:
    une_comments.append(comment)

separator = ', '
une_all_comments = separator.join(une_comments)

# 1 Create the word frequency table
freq_table = _create_frequency_table(une_all_comments)

'''
We already have a sentence tokenizer, so we just need 
to run the sent_tokenize() method to create the array of sentences.
'''

# 2 Tokenize the sentences
sentences = sent_tokenize(une_all_comments)

# 3 Important Algorithm: score the sentences
sentence_scores = _score_sentences(sentences, freq_table)

# 4 Find the threshold
threshold = _find_average_score(sentence_scores)

# 5 Important Algorithm: Generate the summary
summary = _generate_summary(sentences, sentence_scores, 3.0 * threshold)

print('Unemployment Document Summarization')
print('\n',summary)

In [ ]:
irs_comments = []
for comment in irs_data.Comments:
    irs_comments.append(comment)

separator = ', '
irs_all_comments = separator.join(irs_comments)

# 1 Create the word frequency table
freq_table = _create_frequency_table(irs_all_comments)

'''
We already have a sentence tokenizer, so we just need 
to run the sent_tokenize() method to create the array of sentences.
'''

# 2 Tokenize the sentences
sentences = sent_tokenize(irs_all_comments)

# 3 Important Algorithm: score the sentences
sentence_scores = _score_sentences(sentences, freq_table)

# 4 Find the threshold
threshold = _find_average_score(sentence_scores)

# 5 Important Algorithm: Generate the summary
summary = _generate_summary(sentences, sentence_scores, 2.0 * threshold)

print('IRS Document Summarization')
print('\n',summary)

In [ ]:
usps_comments = []
for comment in usps_data.Comments:
    usps_comments.append(comment)

separator = ', '
usps_all_comments = separator.join(usps_comments)

# 1 Create the word frequency table
freq_table = _create_frequency_table(usps_all_comments)

'''
We already have a sentence tokenizer, so we just need 
to run the sent_tokenize() method to create the array of sentences.
'''

# 2 Tokenize the sentences
sentences = sent_tokenize(usps_all_comments)

# 3 Important Algorithm: score the sentences
sentence_scores = _score_sentences(sentences, freq_table)

# 4 Find the threshold
threshold = _find_average_score(sentence_scores)

# 5 Important Algorithm: Generate the summary
summary = _generate_summary(sentences, sentence_scores, 3.5 * threshold)

print('USPS Document Summarization')
print('\n',summary)

# Hypotheses Testing

Unemployment:

Users are:
- dissatisfied with disbursement time/application processing time.
- uncertain of eligibility requirements. 
- frustrated that unemployment pays more than minimum wage.

In [ ]:
# une_h3 = df_une.loc[(df_une['unemployment benefits minimum wage'] == 1)]
# sample = une_h3['Comments'].sample(100)
# #exporting and counting manually
# sample.to_csv('H3 Sample.csv')

une_h1 = df_une.loc[(df_une['eligibility'] == 1)]
# sample = une_h1['Comments'].sample(100)
# sample.to_csv('H1 Sample.csv')
h1_TN = len(df_une.loc[(df_une['eligibility']==0)])
h1_FN = 0

In [ ]:
une_h2 = df_une.loc[(df_une['delayed claims'] == 1)]
# sample = une_h2['Comments'].sample(100)
# sample.to_csv('H2 Sample.csv')
len(une_h2)

In [ ]:
h2_TN = len(df_une.loc[df_une['delayed claims'] == 0])

h2_FN = 0

une_h3 = df_une.loc[(df_une['unemployment benefits minimum wage']==1)]
h3_TN = len(df_une.loc[df_une['unemployment benefits minimum wage']==0])
h3_FN = 0

In [ ]:
!pip install scikit-plot

In [ ]:
from sklearn import metrics
import math
import scikitplot as skplt
import copy

def print_metrics(y_true,y_pred, TN):
    '''As our classifier is more likely to misclassify a comment into a topic than it is to not detect the topic,
    we can safely assume our TN count is equivalent to the sample of 100 from all 18 topics where this topic=0,
    thus, the assumption is made the same ratio of false positives, are also the ratio of false negatives
    the percentage of false positives to N samples is taken and applied to false positives to estimate false negatives'''
    
    conf = metrics.confusion_matrix(y_true, y_pred)
    FP = conf[0][1]
    
    L = math.ceil((FP/len(y_true))*FP)
    
    z = TN - L
    n = np.zeros(L,dtype=np.float64)
    l = np.ones(L,dtype=np.float64)
    negs = np.zeros(z,dtype=np.float64)
    
    y_true = np.concatenate((y_true,negs,l),axis=None)
    y_pred = np.concatenate((y_pred,negs,n),axis=None)
    
    conf = metrics.confusion_matrix(y_true, y_pred)
    
    TP = conf[1][1]
    FP = conf[0][1]
    TN = conf[0][0]
    FN = conf[1][0]
    
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1 = (2*precision)/(precision+recall)
    
    print('Confusion Matrix')
    print('True Negative: ', TN, '\tFalse Positive:', FP, '\nFalse Negative:', FN, '\tTrue Positive:', TP)
    
    print('\nPrecision Score:', precision)
    print('Recall Score:', recall)
    print('F1 Score:',f1)
    
    fpr, tpr, threshold = metrics.roc_curve(y_true, y_pred)

    roc_auc = metrics.auc(fpr, tpr)
    
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
#     plt.xlim([0, 1])
#     plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
h2_y_true = np.array([1,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,0,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,0,1,0,0,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,0,0,1,1,0,1,1,1,1,1],dtype=np.float64)
h2_y_pred = np.ones(100*math.ceil(len(une_h2)/100))
h2_y_true = np.repeat(h2_y_true,math.ceil(len(une_h2)/100))

#extrapolating predictions
print('Hypothesis 2 Evaluation: Delayed Claims')
print_metrics(h2_y_true,h2_y_pred, h2_TN)

In [ ]:
h3_y_true = np.array([1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,1])
h3_y_pred = np.ones(100*math.ceil(len(une_h3)/100))
h3_y_true = np.repeat(h3_y_true,math.ceil(len(une_h3)/100))

#extrapolating predictions
print('Hypothesis 3 Evaluation: Minimum Wage compared to Unemployment')
print_metrics(h3_y_true,h3_y_pred, h3_TN)

In [ ]:
h1_y_true = np.array([1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1],dtype=np.float64)
h1_y_pred = np.ones(100*math.ceil(len(une_h1)/100))
h1_y_true = np.repeat(h1_y_true,math.ceil(len(une_h1)/100))

#extrapolating predictions
print('Hypothesis 1 Evaluation: Eligibility Clarification')
print_metrics(h1_y_true,h1_y_pred, h1_TN)

In [ ]:
excel_une_data = pd.read_csv('excel_une_data.csv',index_col=0)
excel_une_data.head()

In [ ]:
tops = []
for i in excel_une_data['topics']:
    l = i.replace('[', ' ')
    t = l.replace(']', ' ')
    tops.append(t.split(','))
print(tops[0])

In [ ]:
tops = pd.DataFrame(tops)
tops.head()

In [ ]:
tops.to_csv('topics_in_columns.csv')